In [16]:
import numpy as np
import pandas as pd 
import scipy.sparse as ss

from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, f1_score

In [17]:
train_data = pd.read_csv('../data/news_train.csv')
train_data_text_vectors = ss.load_npz('../data/news_train_text_vectors.npz')

test_data = pd.read_csv('../data/news_test.csv')
test_data_text_vectors = ss.load_npz('../data/news_test_text_vectors.npz')

In [18]:
train_data_text_vectors = pd.DataFrame(train_data_text_vectors.toarray())
test_data_text_vectors = pd.DataFrame(test_data_text_vectors.toarray())

In [82]:
# preprocess features
from sklearn.decomposition import PCA
from sklearn import preprocessing
sc = preprocessing.StandardScaler()

def pre_trans(df,vectors):
    pca = PCA(80)
    en = preprocessing.LabelEncoder()
    sc = preprocessing.StandardScaler()
    pt = preprocessing.PolynomialFeatures(degree=2)
    bad_auth = ['Anonymous', 'Alex Ansary', 'Activist Post', 'admin','Henry Wolff']

    vectors = pd.DataFrame(pca.fit_transform(vectors))
    x = df.copy()
    x["fake_auth"] = x.author.apply(lambda x: 0 if x in bad_auth else 1)
    x.published = x.published.apply(lambda x: x[:16])
    x = x.apply(en.fit_transform)
    x = pd.concat([x, vectors], axis=1)
    x = sc.fit_transform(x.values)
    # x = pt.fit_transform(x)
    return x


In [83]:
from sklearn.model_selection import train_test_split
y = train_data.label.replace({"Fake":0, "Real":1})
X = pre_trans(train_data.drop("label", axis=1), train_data_text_vectors)
x_train,x_test , y_train,y_test = train_test_split(X,y, test_size=.2, random_state=313)

In [85]:
# training model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', max_iter=3000)
# model.fit(X, y)
model.fit(x_train, y_train)

LogisticRegression(max_iter=3000)

In [86]:
# evaluate model
from sklearn.metrics import f1_score
y_predict = model.predict(x_test)
f1_score(y_test, y_predict)

0.7058823529411764

In [67]:
x2 = pre_trans(test_data, test_data_text_vectors)
y2 = model.predict(x2)
submission = pd.DataFrame(y2, columns=["label"]).replace({
    0: "Fake",
    1: "Real"
})
submission

,label
0,Real
1,Fake
2,Real
3,Real
4,Fake
...,...
341,Fake
342,Fake
343,Fake
344,Fake
